In [10]:
import pandas as pd
import spacy
import en_core_web_sm
import random

In [5]:
### Procédure de nettoyage ###
data_raw = pd.read_csv('pictures_sample.csv')
data_english = data_raw[data_raw.Language == 'EN']
def author_del(text):
    text_s = text
    while(text_s[-1] != '\n' and len(text_s) > 1): 
        text_s = text_s[:-1]
    if(len(text_s) <= 1):
        return(text)
    return(text_s[:-1])
data_english.Description = data_english.Description.apply(author_del)          
data_english.drop_duplicates(subset='Description', keep='first', inplace=True)
ind = list(data_english.index)
##############################

In [6]:
### Procédure de lecture ###
ent_load = pd.read_pickle("ent.csv")
M = ent_load.to_numpy()
M[M == None] = 0
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        if M[i,j] == 0:
            M[i,j] = []
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        if M[i,j] != []:
            M[i,j] = [M[i,j]]
M = list(M.sum(axis = 1))
scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}
for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]
def dep_to_val(l):
    L = []
    for i in range(len(l)):
        if l[i] in dict_val.keys():
            L.append(dict_val[l[i]])
        else:
            L.append(0)
    return(max(L))
def final(l):
    L = [] 
    for i in l:
        ll = [] 
        for ii in i:
            ll.append((ii[0], dep_to_val(ii[1])))
        L.append(ll)
    for i in range(len(L)):
        name = L[i][0][0]
        score = 0
        for ii in L[i]:
            score += ii[1]
        L[i] = (name, score)
    return(L)
def clean_final(l):
    L = []
    for i in l:
        ll = []
        for ii in i:
            if ii not in ll:
                ll.append(ii)
        L.append(ll)
    return(L)
M = [final(m) for m in M]
M = clean_final(M)
M = [dict(m) for m in M]
############################

In [7]:
# On définit le score entre deux descriptions comme la somme des moyennes des scores pour chaque entité contenue dans les deux
# textes en même temps

def similarity_text(s1, s2):
    score = 0
    for i in s1.keys():
        if i in s2.keys():
            score += (s1[i]+s2[i])/2
    return(score)    

In [27]:
# On affiche le score pour 100 descriptions aléatoires où le score est définit comme 1 si la description associée est la même
# et 0 sinon

r = []

n = 100

for _i in range(n):
    num = random.randint(0, data_english.shape[0])
    while(num in r):
        num = random.randint(0, data_english.shape[0])
    r.append(num)
    
somme = 0

for i in range(n):
    Scores = []
    for j in range(data_english.shape[0]):
        Scores.append(similarity_text(M[r[i]], M[j]))
    if np.argmax(Scores) == r[i]:
        somme += 1
        
score_moyen = somme/n

print(score_moyen)

0.5
